# Simple Vortex in 2D

In [ ]:
import mmf_setup;mmf_setup.nbinit()
%pylab inline --no-import-all
from nbimports import *                # Conveniences like clear_output

Here we generate some vortices.  These are regularized by fixing the coupling constant $g$ so that the homogeneous system in the box and on the lattice gives a fixed value of $\Delta$ at the specified chemical potential.  The self-consistent solution is found by simple iterations.

In [ ]:
from mmf_hfb import bcs, homogeneous;reload(bcs)
from mmfutils.math.special import mstep
from mmfutils.plot import imcontourf

class Vortex(bcs.BCS):
    barrier_width = 0.2
    barrier_height = 100.0
    
    def __init__(self, Nxyz=(32, 32), Lxyz=(10., 10.), **kw):
        self.R = min(Lxyz)/2
        bcs.BCS.__init__(self, Nxyz=Nxyz, Lxyz=Lxyz, **kw)
    
    def get_v_ext(self):
        r = np.sqrt(sum([_x**2 for _x in self.xyz[:2]]))
        R0 = self.barrier_width * self.R
        V = self.barrier_height * mstep(r-self.R+R0, R0)
        return (V, V)
    
def to3d(x, flag=False):
    """convert a 2d data to 3d data
        Arguments
    ---------
    x the original data
    For example:
    [[1,2],[3,4]]
    flag = False
    return [[[0,1],[0,2],[[0,3],[0,4]]]
    flag = True
    return [[[1,0],[2,0],[[3,0],[4,0]]]
   """
    dx, dy = x.shape
    data = []
    if flag:
        for i in range(dx):
            r = []
            for j in range(dy):
                r.append([x[j][i],0])
            data.append(r)
        return data
    
    for i in range(dx):
        r = []
        for j in range(dy):
            r.append([0,x[j][i]])
        data.append(r)
    return data

class VortexState(Vortex):
    def __init__(self, mu, dmu, delta, **kw):
        Vortex.__init__(self, **kw)
        self.mus = (mu+dmu, mu-dmu)
        self.g = self.get_g(mu=mu, delta=delta)
        x, y = self.xyz
        self.Delta = delta*(x+1j*y)
        
    def get_g(self, mu=1.0, delta=0.2):
        h = homogeneous.Homogeneous(Nxyz=self.Nxyz, Lxyz=self.Lxyz) 
        res = h.get_densities(mus_eff=(mu, mu), delta=delta)
        g = delta/res.nu.n
        return g
    
    def solve(self, tol=0.01, plot=True):
        err = 1.0
        fig = None
        with NoInterrupt() as interrupted:
            display(self.plot())
            clear_output(wait=True)

            while not interrupted and err > tol:
                res = self.get_densities(mus_eff=self.mus, delta=self.Delta)
                Delta0, self.Delta = self.Delta, self.g*res.nu  # ....
                err = abs(Delta0 - self.Delta).max()
                if display:
                    plt.clf()
                    fig = self.plot(fig=fig, res=res)
                    plt.suptitle(f"err={err}")
                    display(fig)
                    clear_output(wait=True)
        
    
    def plot(self, fig=None, res=None):
        x, y = self.xyz
        if fig is None:
            fig = plt.figure(figsize=(20, 10))
        plt.subplot(231)
        imcontourf(x, y, abs(self.Delta), aspect=1)
        plt.title(r'$|\Delta|$'); plt.colorbar()
        
        if res is not None:
            plt.subplot(232)
            
            imcontourf(x, y, (res.n_a+res.n_b).real, aspect=1)
            plt.title(r'$n_+$'); plt.colorbar()
        
            plt.subplot(233)
            imcontourf(x, y, (res.n_a-res.n_b).real, aspect=1)
            plt.title(r'$n_-$'); plt.colorbar()
            
            plt.subplot(234)
            
            j_a = res.j_a[0] + 1j*res.j_a[1]
            j_b = res.j_b[0] + 1j*res.j_b[1]
            j_p = j_a + j_b
            j_m = j_a - j_b
            #j_a = (np.cross(to3d(res.j_a[0]), to3d(res.j_a[1], True)))
            #j_b = (np.cross(to3d(res.j_b[0]), to3d(res.j_b[1], True)))
            utheta = np.exp(1j*np.angle(x + 1j*y))
            #imcontourf(x, y, -(j_p*utheta).real, aspect=1)
            imcontourf(x, y, abs(j_a), aspect=1)
            plt.title(r'$j_a$'); plt.colorbar()
            plt.quiver(x.ravel(), y.ravel(), j_a.real.T, j_a.imag.T)
            
            plt.subplot(235)
            #imcontourf(x, y, (j_a - j_b).real, aspect=1)
            #imcontourf(x, y, abs(j_m), aspect=1)
            imcontourf(x, y, abs(j_m), aspect=1)
            #imcontourf(x, y, -(j_m*utheta).real, aspect=1)
            plt.title(r'$J_-$'); plt.colorbar()
            plt.quiver(x.ravel(), y.ravel(), j_m.real.T, j_m.imag.T)
        return fig      

v = Vortex(Nxyz=(16, 16))

$$
  v_x +\I v_y = v e^{\I\phi}, \\
  \uvect{\theta} = \frac{\I x - y}{r}\\
  \uvect{\theta}\cdot \vect{v} = \frac{-yv_x + xv_y}{r} = - \Re(v \uvect{\theta})
$$

# Potential Profile

In [ ]:
V = v.get_v_ext()[0]
x , y = v.xyz
imcontourf(x, y, V, aspect=1);plt.colorbar()

# Symmetric Vortex

In [ ]:
v = VortexState(mu=10.0, dmu=0.0, delta=1.0, Nxyz=(32, 32))
v.solve(plot=True)

# Polarized Vortex

In [ ]:
v = VortexState(mu=10.0, dmu=2.0, delta=5.0, Nxyz=(32, 32))
v.solve(plot=True)

# Compare to FF State

In [ ]:
from mmf_hfb import FuldeFerrelState; reload(FuldeFerrelState)
N = v.Nxyz[0]
L = v.Lxyz[0]
k_c = np.sqrt(2)*np.pi*N/L
mu = 10.0
dmu = 2.0
delta = 5.0
args = dict(mu=mu, dmu=dmu, delta=delta, k_c=k_c)
f = FuldeFerrelState.FFState(fix_g=True, **args)
rs = np.linspace(0.01,1, 10)
rs = np.append(rs, np.linspace(1.1, 3, 15))
ds = [f.solve(r=_r, a=0.001, b=2*delta) for _r in rs]

In [ ]:
r = np.sqrt(sum(_x**2 for _x in v.xyz))
plt.plot(r.ravel(), abs(v.Delta).ravel(), '+')
plt.plot(rs, ds)

In [ ]:
na = np.array([])
nb = np.array([])
res = v.get_densities(mus_eff=v.mus, delta=v.Delta)
for i in range(len(rs)):
    na_, nb_ = f.get_densities(delta=ds[i], r=rs[i], mu_a=mu+dmu, mu_b=mu-dmu)
    na = np.append(na, na_.n)
    nb = np.append(nb, nb_.n)   

In [ ]:
plt.figure(figsize=(20, 5))
plt.subplot(121)
n_p = na + nb
plt.plot(r.ravel(), abs(res.n_a + res.n_b).ravel(), '+')
plt.plot(rs, n_p)
plt.xlabel("r", fontsize=20), plt.ylabel(r"$n_p$", fontsize=20)
plt.title("Total Density")

plt.subplot(122)
n_m = na - nb
plt.plot(r.ravel(), abs(res.n_a - res.n_b).ravel(), 'o')
plt.plot(rs, n_m)
plt.xlabel("r", fontsize=20), plt.ylabel(r"$n_m$", fontsize=20),plt.title("Density Difference")
clear_output()

In [ ]:
ja = np.sqrt(res.j_a[0]**2 + res.j_a[1]**2)
jb = np.sqrt(res.j_b[0]**2 + res.j_b[1]**2) 
j_p_, j_m_ = ja + jb, ja - jb
ja = []
jb = []
for i in range(len(rs)):
    ja.append(na[i]/rs[i])
    jb.append(nb[i]/rs[i])
ja, jb = np.array(ja), np.array(jb)
j_p, j_m = ja + jb, ja - jb
plt.figure(figsize=(20, 5))
plt.subplot(121)
plt.plot(r.ravel(), j_p_.ravel(), 'o')
plt.plot(rs, j_p)
plt.xlabel("r", fontsize=20), plt.ylabel(r"$j_p$", fontsize=20),plt.title("Total Current")
plt.subplot(122)
plt.plot(r.ravel(), j_m_.ravel(), 'o')
plt.plot(rs, j_m)
plt.xlabel("r", fontsize=20), plt.ylabel(r"$j_m$", fontsize=20),plt.title("Current Difference")
clear_output()


$$
  (x - \I y)^2 = r^2e^{-2\I \theta}
$$

# Quiver

In [ ]:
x, y = VortexState(mu=10.0, dmu=0.4, delta=1.0, Nxyz=(32, 32)).xyz
v = 1j*x - 2*y
plt.quiver(x.ravel(), y.ravel(), v.T.real, v.T.imag)

$$
  -u^\dagger \I\nabla u
$$